In [1]:
import sys
print(sys.executable)

/opt/anaconda3/bin/python


In [2]:
import pandas as pd
import seaborn as sns
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [4]:
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from bs4 import BeautifulSoup

In [5]:
import os

In [6]:
df = pd.read_csv("imdb.csv")

In [7]:
df.head

<bound method NDFrame.head of                                                   review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]>

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

In [9]:
train.head

<bound method NDFrame.head of                                                   review sentiment
46040  Contrary to popular belief, this title , to me...  positive
16504  This is one of the many Techno-phobic movies t...  positive
23698  An old man is riding his bike down a village r...  positive
24244  I watched this movie in the wee hours of the m...  positive
46272  Maybe I expected too much of this film, but at...  negative
...                                                  ...       ...
48446  I recently saw this at the 2007 Palm Springs I...  positive
31292  This is an extraordinary film. As a courtroom ...  positive
11378  Well...overall, this movie was pretty much wor...  negative
34854  Having heard so many people raving about this ...  negative
45776  If I could give it a zero, I'd change my mind ...  negative

[40000 rows x 2 columns]>

preprocessing

In [10]:
ptrain = train[train.sentiment == "positive"]

In [11]:
ntrain = train[train.sentiment == "negative"]

In [12]:
ntrain.head

<bound method NDFrame.head of                                                   review sentiment
46272  Maybe I expected too much of this film, but at...  negative
24665  This movie has to be the worst film of 2007, i...  negative
33314  Shaggy & Scooby-Doo Get a Clue. It's like watc...  negative
39979  TDY is probably the single worst piece of tras...  negative
42909  As a Turkish man now living in Sweden I must c...  negative
...                                                  ...       ...
13375  Nothing can prepare you for another lousy bimb...  negative
44043  Who actually created this piece of crap this i...  negative
11378  Well...overall, this movie was pretty much wor...  negative
34854  Having heard so many people raving about this ...  negative
45776  If I could give it a zero, I'd change my mind ...  negative

[20026 rows x 2 columns]>

In [13]:
ptrain.head

<bound method NDFrame.head of                                                   review sentiment
46040  Contrary to popular belief, this title , to me...  positive
16504  This is one of the many Techno-phobic movies t...  positive
23698  An old man is riding his bike down a village r...  positive
24244  I watched this movie in the wee hours of the m...  positive
18075  It's a funny business, reviewing movies. These...  positive
...                                                  ...       ...
32860  I swear if I did ever tried cocaine I'd be abl...  positive
258    Black comedy isn't always an easy sell. Every ...  positive
37557  I'm sick and tired of people complaining that ...  positive
48446  I recently saw this at the 2007 Palm Springs I...  positive
31292  This is an extraordinary film. As a courtroom ...  positive

[19974 rows x 2 columns]>

In [ ]:
negativereviews = []
for review in ntrain["review"]:
    review = review.replace("<br />", " ")
    doc = nlp(review)
    tokenbyreview = []
    ##print(review)
    for token in doc:
        if token.is_stop==False and token.is_punct==False and token.text.isnumeric()==False:
            tokenbyreview.append(token.text.lower())
            ##print(token.text.lower())
    ##print(tokenbyreview)
    negativereviews.append(set(tokenbyreview))

In [ ]:
##negativereviews

In [ ]:
count_negativetrain = {}
for bag in negativereviews:
    for word in bag:
        if word not in count_negativetrain:
            count_negativetrain[word]=1
        else: 
            count_negativetrain[word]+=1
##print(sorted(list(count_negativetrain.items()), key=lambda x: x[1], reverse=True)) 

In [ ]:
positivereviews = []
for review in ptrain["review"]:
    review = review.replace("<br />", " ")
    doc = nlp(review)
    tokenbypreview = []
    ##print(review)
    for token in doc:
        if token.is_stop==False and token.is_punct==False and token.text.isnumeric()==False:
            tokenbypreview.append(token.text.lower())
            ##print(token.text.lower())
    ##print(tokenbyreview)
    positivereviews.append(set(tokenbypreview))

In [ ]:
##positivereviews

In [ ]:
nlp.pipe_names

In [ ]:
count_positivetrain = {}
for bag in positivereviews:
    for word in bag:
        if word not in count_positivetrain:
            count_positivetrain[word]=1
        else: 
            count_positivetrain[word]+=1
##print(sorted(list(count_positivetrain.items()), key=lambda x: x[1], reverse=True)) 

In [ ]:
numtrue_positive = 0
numtrue_negative = 0
numfalse_positive = 0
numfalse_negative = 0
for _,row in test.iterrows():
    review = row["review"]
    ##review = "it was the best. loved it."
    review = review.replace("<br />", " ")
    doc = nlp(review)
    tokenbyreview = []
    ##print(review)
    for token in doc:
        if token.is_stop==False and token.is_punct==False and token.text.isnumeric()==False:
            tokenbyreview.append(token.text.lower())
    tokenbyreview = set(tokenbyreview)
    score = 0
    for word in tokenbyreview: 
        if word in count_positivetrain: 
            score+=count_positivetrain[word]
        if word in count_negativetrain: 
            score-=count_negativetrain[word]
    ##print(score)
    if (score>0 and row["sentiment"] == "positive"): numtrue_positive+=1
    if (score<0 and row["sentiment"] == "negative"): numtrue_negative+=1
    if (score<0 and row["sentiment"] == "positive"): numfalse_negative+=1
    if (score>0 and row["sentiment"] == "negative"): numfalse_positive+=1    
print(f"Number of true positives: {numtrue_positive}, or {numtrue_positive/len(test)*100}%")
print(f"Number of true negatives: {numtrue_negative}, or {numtrue_negative/len(test)*100}%")
print(f"Number of false positives: {numfalse_positive}, or {numfalse_positive/len(test)*100}%")
print(f"Number of false negatives: {numfalse_negative}, or {numfalse_negative/len(test)*100}%")
accuracy = (numtrue_positive+numtrue_negative)/len(test)
print(accuracy)